In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (806/806), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 9809 (delta 741), reused 711 (delta 711), pack-reused 9003
Receiving objects: 100% (9809/9809), 9.50 MiB | 21.28 MiB/s, done.
Resolving deltas: 100% (7284/7284), done.
/content/deep-person-reid
     |████████████████████████████████| 5.8 MB 5.3 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 185 kB 46.4 MB/s 
     |████████████████████████████████| 42 kB 911 kB/s 
     |████████████████████████████████| 42 kB 856 kB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 73 kB 1.5 MB/s 
     |████████████████████████████████| 68 kB 6.1 MB/s 
     |████████████████████████████████| 41 kB 532 kB/s 
/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly reco

In [ ]:
import torchreid

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3392 KB/s, 43 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
 model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.468 (0.770)	data 0.000 (0.286)	eta 1:43:35	loss 6.9469 (6.7742)	acc 3.1250 (0.6250)	lr 0.000300
epoch: [1/20][20/404]	time 0.475 (0.621)	data 0.002 (0.143)	eta 1:23:22	loss 6.7817 (6.9180)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/20][30/404]	time 0.468 (0.570)	data 0.001 (0.096)	eta 1:16:28	loss 6.6605 (6.8471)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][40/404]	time 0.471 (0.545)	data 0.001 (0.072)	eta 1:12:58	loss 6.6874 (6.7834)	acc 0.0000 (0.5469)	lr 0.000300
epoch: [1/20][50/404]	time 0.470 (0.529)	data 0.000 (0.058)	eta 1:10:51	loss 6.5498 (6.7464)	acc 0.0000 (0.5000)	lr 0.000300
epoch: [1/20][60/404]	time 0.467 (0.519)	data 0.002 (0.049)	eta 1:09:23	loss 6.6834 (6.7251)	acc 0.0000 (0.4167)	lr 0.000300
epoch: [1/20][70/404]	time 0.471 (0.512)	data 0.001 (0.042)	eta 1:08:17	loss 6.6329 (6.7030)	acc 0.0000 (0.3571)	lr 0.000300
epoch: [1/20][80/404]	time 0.470 (0.506)	data 0.000 (0.037)	eta 1:07:28	loss 6.5199 (6.6932)	acc 0.0000 (0.3516)	lr 0.000300


In [ ]:
 model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 39.3MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
engine.run(
    save_dir='log/OSNet',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.495 (0.616)	data 0.000 (0.104)	eta 1:22:52	loss 6.6249 (6.6450)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/20][20/404]	time 0.494 (0.555)	data 0.000 (0.052)	eta 1:14:36	loss 6.5863 (6.6088)	acc 3.1250 (0.3125)	lr 0.000300
epoch: [1/20][30/404]	time 0.494 (0.536)	data 0.001 (0.036)	eta 1:11:51	loss 6.6051 (6.5979)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][40/404]	time 0.497 (0.525)	data 0.000 (0.027)	eta 1:10:22	loss 6.3425 (6.5944)	acc 12.5000 (1.0156)	lr 0.000300
epoch: [1/20][50/404]	time 0.496 (0.519)	data 0.001 (0.022)	eta 1:09:29	loss 6.4109 (6.5733)	acc 3.1250 (1.1875)	lr 0.000300
epoch: [1/20][60/404]	time 0.503 (0.516)	data 0.000 (0.018)	eta 1:08:56	loss 6.3489 (6.5448)	acc 0.0000 (1.4583)	lr 0.000300
epoch: [1/20][70/404]	time 0.494 (0.513)	data 0.001 (0.016)	eta 1:08:29	loss 6.3829 (6.5224)	acc 3.1250 (1.6964)	lr 0.000300
epoch: [1/20][80/404]	time 0.499 (0.511)	data 0.000 (0.014)	eta 1:08:07	loss 6.1800 (6.4817)	acc 3.1250 (2.1094)	lr 0.000300

In [ ]:
model = torchreid.models.build_model(
    name='osnet_ain_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 152MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [ ]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.474 (0.788)	data 0.001 (0.294)	eta 1:46:03	loss 6.5441 (6.6045)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][20/404]	time 0.474 (0.633)	data 0.000 (0.148)	eta 1:25:00	loss 6.4501 (6.5932)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/20][30/404]	time 0.473 (0.581)	data 0.001 (0.099)	eta 1:17:58	loss 6.5377 (6.5733)	acc 0.0000 (1.1458)	lr 0.000300
epoch: [1/20][40/404]	time 0.477 (0.555)	data 0.001 (0.074)	eta 1:14:22	loss 6.5343 (6.5733)	acc 0.0000 (1.0156)	lr 0.000300
epoch: [1/20][50/404]	time 0.483 (0.540)	data 0.000 (0.060)	eta 1:12:14	loss 6.4370 (6.5548)	acc 3.1250 (1.1875)	lr 0.000300
epoch: [1/20][60/404]	time 0.474 (0.530)	data 0.001 (0.050)	eta 1:10:49	loss 6.7955 (6.5303)	acc 0.0000 (1.5104)	lr 0.000300
epoch: [1/20][70/404]	time 0.478 (0.523)	data 0.001 (0.043)	eta 1:09:51	loss 6.3759 (6.5032)	acc 0.0000 (1.9196)	lr 0.000300
epoch: [1/20][80/404]	time 0.480 (0.518)	data 0.001 (0.038)	eta 1:09:01	loss 6.4474 (6.4640)	acc 6.2500 (2.4609)	lr 0.000300
